In [3]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import holidays
import os
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns 
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager,rc
import platform
# 한글 폰트 설정
if platform.system() == 'Windows':# 윈도우인 경우
    font_path = "c:/Windows/Fonts/malgun.ttf" # 맑은 고딕 폰트 경로
    font_name = font_manager.FontProperties(fname=font_path).get_name()
    plt.rc('font', family=font_name)
# 음수 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

In [4]:
data_path = "C:/Users/choin/Downloads/FitBit"

In [30]:
heart = pd.read_csv(os.path.join(data_path, 'heartrate_seconds_merged.csv'))
heart.head()

,Id,Time,Value
0,2022484408,4/12/2016 7:21:00 AM,97
1,2022484408,4/12/2016 7:21:05 AM,102
2,2022484408,4/12/2016 7:21:10 AM,105
3,2022484408,4/12/2016 7:21:20 AM,103
4,2022484408,4/12/2016 7:21:25 AM,101


In [18]:
heart.columns

Index(['Id', 'Time', 'Value'], dtype='object')

In [32]:
# DataFrame 생성

# 'Time'을 datetime 형식으로 변환
heart['Time'] = pd.to_datetime(heart['Time'])

# 'Time'을 인덱스로 설정
heart.set_index('Time', inplace=True)

# 'Time_diff' 계산: 각 시간 차이 (초 단위)
heart['Time_diff'] = heart.index.to_series().diff().dt.total_seconds()

# 'Time_diff'의 첫 번째 값은 NaN이므로, 이를 0으로 설정
heart['Time_diff'].fillna(0, inplace=True)

# 'Value.diff()'로 심박수 차이를 구하고, 이를 'Time_diff'로 나누어 RR 간격 계산
# 첫 번째 행에서 RR 간격은 계산하지 않도록 처리
heart['RR_interval'] = heart['Time_diff'] / heart['Value'].diff().abs()

# RR_interval에서 무한대(inf)나 음수 값은 NaN으로 처리
heart['RR_interval'].replace([np.inf, -np.inf], np.nan, inplace=True)

# 1분 단위로 리샘플링하여 평균 계산
reheart = heart.resample('T').agg({
    'Value': 'mean',         # 1분 간격으로 심박수 평균
    'RR_interval': 'mean'    # 1분 간격으로 RR 간격 평균
})

# 'Time'을 다시 컬럼으로 추가
reheart.reset_index(inplace=True)

# 결과 출력
reheart.head()

C:\Users\choin\AppData\Local\Temp\ipykernel_6684\3597048254.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  heart['Time_diff'].fillna(0, inplace=True)
C:\Users\choin\AppData\Local\Temp\ipykernel_6684\3597048254.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

,Time,Value,RR_interval
0,2016-04-12 00:00:00,61.475000,-74061.393939
1,2016-04-12 00:01:00,64.024390,7.187500
2,2016-04-12 00:02:00,64.157895,5.740741
3,2016-04-12 00:03:00,64.083333,10.625000
4,2016-04-12 00:04:00,63.888889,9.166667


In [28]:
# 'Time'을 datetime 형식으로 변환
heart['Time'] = pd.to_datetime(heart['Time'])

# 'Time'을 인덱스로 설정
heart.set_index('Time', inplace=True)

# RR 간격 계산: 시간 차이 (초 단위)
heart['Time_diff'] = heart.index.to_series().diff().dt.total_seconds()
heart['Time_diff'].fillna(0, inplace=True)

# RR 간격 계산 (초 단위): 심박수 차이를 시간 차이로 나누기
# Time_diff가 0인 경우와 심박수 차이가 0인 경우 처리
heart['RR_interval'] = heart['Time_diff'] / heart['Value'].diff().abs()

# 심박수 차이가 0인 경우(즉, 심박수 값이 변하지 않은 경우)는 RR_interval을 NaN으로 처리
heart['RR_interval'].replace([np.inf, -np.inf], np.nan, inplace=True)

# 1분 단위로 리샘플링하여 평균 계산
reheart = heart.resample('T').agg({
    'Value': 'mean',         # 1분 간격으로 심박수 평균
    'RR_interval': 'mean'    # 1분 간격으로 RR 간격 평균
})

# 결과 출력
print(reheart.head())

C:\Users\choin\AppData\Local\Temp\ipykernel_6684\1453457863.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  heart['Time_diff'].fillna(0, inplace=True)
C:\Users\choin\AppData\Local\Temp\ipykernel_6684\1453457863.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w

                         Value   RR_interval
Time                                        
2016-04-12 00:00:00  61.475000 -74061.393939
2016-04-12 00:01:00  64.024390      7.187500
2016-04-12 00:02:00  64.157895      5.740741
2016-04-12 00:03:00  64.083333     10.625000
2016-04-12 00:04:00  63.888889      9.166667


In [29]:
reheart.columns

Index(['Value', 'RR_interval'], dtype='object')

In [12]:
# 'Time'을 datetime 형식으로 변환
heart['Time'] = pd.to_datetime(heart['Time'])

# 1분 단위로 리샘플링하여 평균 심박수 계산
heart.set_index('Time', inplace=True)  # Time 컬럼을 인덱스로 설정
reheart = heart.resample('T').mean()  # 1분 단위('T'는 Minute)의 평균

# 결과 출력
print(reheart)

C:\Users\choin\AppData\Local\Temp\ipykernel_6684\2161214270.py:6: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  reheart = heart.resample('T').mean()  # 1분 단위('T'는 Minute)의 평균


                               Id      Value
Time                                        
2016-04-12 00:00:00  4.778084e+09  61.475000
2016-04-12 00:01:00  4.936888e+09  64.024390
2016-04-12 00:02:00  4.932441e+09  64.157895
2016-04-12 00:03:00  4.939869e+09  64.083333
2016-04-12 00:04:00  4.858151e+09  63.888889
...                           ...        ...
2016-05-12 16:16:00  4.558610e+09  75.222222
2016-05-12 16:17:00  4.558610e+09  74.250000
2016-05-12 16:18:00  4.558610e+09  73.714286
2016-05-12 16:19:00  4.558610e+09  75.500000
2016-05-12 16:20:00  4.558610e+09  76.000000

[44181 rows x 2 columns]


In [22]:
heart['Time'] = pd.to_datetime(heart['Time'])

# RR 간격 계산: (시간 차이) / (심박수 차이)
heart['Time_diff'] = heart['Time'].diff().dt.total_seconds()  # 시간 차이 (초 단위)
heart['HR_diff'] = heart['Value'].diff()  # 심박수 차이

# RR 간격 계산 (단위: 초)
heart['RR_interval'] = heart['Time_diff'] / heart['HR_diff'].abs()  # RR 간격 (단위: 초)

# 심박수 차이가 0인 경우(0으로 나누는 경우) 처리
heart['RR_interval'] = heart['RR_interval'].replace([np.inf, -np.inf], np.nan)  # inf 값은 NaN으로 바꾸기

# 첫 번째 행의 NaN 처리: 첫 번째 행 삭제 또는 다른 값으로 채우기 (여기서는 첫 번째 행 삭제)
heart = heart.dropna(subset=['RR_interval'])

# HRV 지표 계산: RMSSD, SDNN
rmssd = np.sqrt(np.mean(np.square(np.diff(heart['RR_interval']))))  # RMSSD
sdnn = np.std(heart['RR_interval'])  # SDNN

print(f"RMSSD: {rmssd}")
print(f"SDNN: {sdnn}")

# 결과 출력 (초 단위 데이터)
print(heart)

RMSSD: 2671.5980525827117
SDNN: 1889.1219827384684
                 Id                Time  Value  Time_diff  HR_diff  \
1        2022484408 2016-04-12 07:21:05    102        5.0      5.0   
2        2022484408 2016-04-12 07:21:10    105        5.0      3.0   
3        2022484408 2016-04-12 07:21:20    103       10.0     -2.0   
4        2022484408 2016-04-12 07:21:25    101        5.0     -2.0   
5        2022484408 2016-04-12 07:22:05     95       40.0     -6.0   
...             ...                 ...    ...        ...      ...   
2483652  8877689391 2016-05-12 14:43:38     58       15.0     -1.0   
2483653  8877689391 2016-05-12 14:43:53     57       15.0     -1.0   
2483654  8877689391 2016-05-12 14:43:58     56        5.0     -1.0   
2483655  8877689391 2016-05-12 14:44:03     55        5.0     -1.0   
2483657  8877689391 2016-05-12 14:44:28     56       10.0      1.0   

         RR_interval  
1           1.000000  
2           1.666667  
3           5.000000  
4           2.50